In [31]:
from tensorflow.keras.layers import Input, Dropout, GlobalAveragePooling2D, Dense, LeakyReLU, Conv2D, MaxPooling2D,\
      Flatten, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, Callback
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Model
from timeit import default_timer as timer
from tensorflow.keras.applications.inception_v3 import InceptionV3
import numpy as np
from sklearn.metrics import classification_report
import const
import os


In [32]:
#Define dataset path and Timingcallback class

dataset_used = const.DATASET_REDUIT

diverted_images_train_path = os.path.join(dataset_used, "train")
diverted_images_valid_path = os.path.join(dataset_used, "valid")
diverted_images_test_path = os.path.join(dataset_used, "test")

class TimingCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

In [ ]:
#Define callbacks objects
reduce_learning_rate = ReduceLROnPlateau(monitor="val_loss", patience=2, min_delta=0.01, factor=0.1, cooldown=4, verbose=1)
early_stopping = EarlyStopping(patience=3, min_delta=0.01, verbose=1, mode='min', monitor='val_loss')
time_callback = TimingCallback()

In [ ]:
#Define building moodel functions
def build_model_dense(nb_class, dense_activation='relu', dropout_rate=0.2, dense_kernel_regularizer = None,
                unit_first_dense=1024):
    input_layer = Input(shape=(224, 224, 3))
    x = GlobalAveragePooling2D()(input_layer)
    x = Dense(unit_first_dense, activation=dense_activation)(x)
    x = Dropout(rate=dropout_rate)(x)
    x = Dense(unit_first_dense/2, activation=dense_activation)(x)
    x = Dropout(rate=dropout_rate)(x)
    predictions = Dense(nb_class, activation='softmax')(x)
    model = Model(inputs=input_layer, outputs=predictions)
    return model

def build_model_cnn(nb_class, conv2D_activation='relu'):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation=conv2D_activation, input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), padding='same', activation=conv2D_activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same', activation=conv2D_activation))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), padding='same', activation=conv2D_activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(128, (3, 3), padding='same', activation=conv2D_activation))
    model.add(BatchNormalization())
    model.add(Conv2D(128, (3, 3), padding='same', activation=conv2D_activation))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(nb_class, activation='softmax'))

    return model

In [ ]:
#First try with dense model
batch_size = 16

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

nb_class = train_generator.num_classes

model = build_model_dense(nb_class)
#model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#Increase batch size and change Adam learning rate
batch_size = 20

train_generator = ImageDataGenerator().flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

nb_class = train_generator.num_classes

model = build_model_dense(nb_class)
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#Change dropout rate
batch_size = 20

valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

nb_class = train_generator.num_classes

model = build_model_dense(nb_class, dropout_rate=0.5)
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#Change optimizer from Adam to SGD, add kernel regularizer and change unit value for denses layers
batch_size = 20

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), batch_size=batch_size)
valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

nb_class = train_generator.num_classes

model = build_model_dense(nb_class, dropout_rate=0.5, dense_kernel_regularizer=l2(0.01), unit_first_dense=2048)
#model.summary()

optimizer = SGD(learning_rate=0.01) 
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

# Entraînement du modèle
history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
# First try with a simple CNN model
batch_size = 20

valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

nb_class = train_generator.num_classes
def build_model_cnn(nb_class, conv2d_activation='relu'):
    input_layer = Input(shape=(224, 224, 3))
    x = Conv2D(32, (3, 3), activation=conv2d_activation)(input_layer)  
    x = MaxPooling2D(pool_size=(2, 2))(x) 
    x = Conv2D(64, (3, 3), activation=conv2d_activation)(x)  
    x = MaxPooling2D(pool_size=(2, 2))(x)  
    x = Flatten()(x) 
    x = Dense(128, activation='relu')(x)  
    predictions = Dense(nb_class, activation='softmax')(x)  
    model = Model(inputs=input_layer, outputs=predictions)
    return model

model = build_model_cnn(nb_class)
#model.summary()

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=5,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
#Complexification of CNN model

batch_size = 20

valid_generator = ImageDataGenerator().flow_from_directory(diverted_images_valid_path, target_size=(224, 224), batch_size=batch_size)
test_generator = ImageDataGenerator().flow_from_directory(diverted_images_test_path, target_size=(224, 224), batch_size=batch_size)

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
    diverted_images_train_path,
    target_size=(224, 224),
    batch_size=batch_size)

nb_class = train_generator.num_classes

model = build_model_cnn(nb_class)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=10,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])


In [ ]:
# Try with data augmentation (best model)
batch_size = 16
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(diverted_images_train_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
valid_generator = valid_datagen.flow_from_directory(diverted_images_valid_path, target_size=(224, 224), 
                                                    batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(diverted_images_test_path, target_size=(224, 224), 
                                                  batch_size=batch_size, class_mode='categorical')

nb_class = train_generator.num_classes

model = build_model_cnn(nb_class)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc', 'mean_absolute_error'])

history_diverted = model.fit(train_generator,
                    epochs=20,
                    steps_per_epoch=train_generator.samples//train_generator.batch_size,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//valid_generator.batch_size,
                    callbacks=[reduce_learning_rate, early_stopping, time_callback], verbose=1)

test_accuracy = model.evaluate(test_generator)
print(test_accuracy)
print(history_diverted.history['val_acc'][-1])
